In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json, os

# ===== PATH =====
DATASET_PATH = "traning"
MODEL_DIR = "model"
MODEL_PATH = "model/uang_mobilenet_lite.h5"
CLASS_PATH = "model/class_indices.json"

IMG_SIZE = (160, 160)   # lebih kecil = lebih ringan
BATCH_SIZE = 16
EPOCHS = 100

os.makedirs(MODEL_DIR, exist_ok=True)

# ===== DATA =====
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

with open(CLASS_PATH, "w") as f:
    json.dump(train_data.class_indices, f)

# ===== MODEL =====
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    alpha=0.35,               # 🔥 super ringan
    input_shape=(160,160,3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=Adam(0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ===== TRAIN =====
model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

# ===== SAVE =====
model.save(MODEL_PATH)
print(f"✅ Model ringan tersimpan: {MODEL_PATH}")


Found 896 images belonging to 7 classes.
Found 224 images belonging to 7 classes.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 160, 160, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 80, 80, 16)        │             432 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 80, 80, 16)        │              64 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 80, 80, 16)        │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 80, 80, 16)        │             144 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 80, 80, 16)        │              64 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 80, 80, 16)        │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 80, 80, 8)         │             128 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 80, 80, 8)         │              32 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 80, 80, 48)        │             384 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 80, 80, 48)        │             192 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 80, 80, 48)        │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 81, 81, 48)        │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 40, 40, 48)        │             432 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 419,175 (1.60 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 410,208 (1.56 MB)

Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 73s 885ms/step - accuracy: 0.3237 - loss: 1.7965 - val_accuracy: 0.4598 - val_loss: 1.5229
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 403ms/step - accuracy: 0.5692 - loss: 1.3027 - val_accuracy: 0.5938 - val_loss: 1.2655
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 429ms/step - accuracy: 0.6908 - loss: 1.0458 - val_accuracy: 0.6607 - val_loss: 1.0907
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 410ms/step - accuracy: 0.7656 - loss: 0.8767 - val_accuracy: 0.7054 - val_loss: 0.9702
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 22s 399ms/step - accuracy: 0.7701 - loss: 0.7753 - val_accuracy: 0.7366 - val_loss: 0.8594
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 44s 447ms/step - accuracy: 0.8471 - loss: 0.6644 - val_accuracy: 0.7679 - val_loss: 0.7996
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 426ms/step - accuracy: 0.8527 - loss: 0.6067 - val_accuracy: 0.7812 - val_loss: 0.7506
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 400ms/step - accuracy: 0.8672 - loss: 0.5491 - 

✅ Model ringan tersimpan: model/uang_mobilenet_lite.h5
